# Projet Programmation GPU

*Par DE CASTRO Lucas et LATEB Samy*

## Introduction

Dans ce notebook, nous présentons une solution de calcul numérique d'intégrales optimisée en utilisant CUDA (Compute Unified Device Architecture), une plateforme de calcul parallèle développée par NVIDIA. Notre objectif est de tirer parti de la puissance de calcul massive offerte par les GPU, offrant ainsi des gains significatifs en termes de temps d'exécution par rapport aux approches séquentielles traditionnelles sur CPU.

Nous élaborerons d'abord une version de notre programme en C++ classique afin de pouvoir le comparer à notre implémentation CUDA par la suite, mais également pour montrer la logique de calcul de nos intégrales.

# Prerequis

Dans cette partie, nous allons verifier que tous les éléments nécessaires pour utiliser CUDA sont fonctionnels dans ce Notebook Google Colab.




In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!nvidia-smi

Fri Mar 15 08:29:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Basic C++ implementation

In [ ]:
%%writefile cpu.cpp
#include <iostream>
#include <cstdlib>

// Polynomial definition here
double polynomial(double x) {
    return x;
}

// Integration logic using a basic trapeizodial numeric estimation
double computeIntegral(double a, double b, int num_intervals) {
    double h = (b - a) / num_intervals;
    double sum = 0.5 * (polynomial(a) + polynomial(b));

    for (int i = 1; i < num_intervals; i++) {
        double x = a + i * h;
        sum += polynomial(x);
    }

    return sum * h;
}

int main(int argc, char* argv[]) {
    if (argc < 4) {
        std::cerr << "Usage: " << argv[0] << " <a> <b> <num_interval>" << std::endl;
        return 1;
    }

    double a = std::atof(argv[1]);
    double b = std::atof(argv[2]);
    int num_intervals = std::atoi(argv[3]);

    double integral = computeIntegral(a, b, num_intervals);

    std::cout << "The integral of the polynomial in the range [" << a << ", " << b << "] is: " << integral << std::endl;

    return 0;
}

Overwriting cpu.cpp


In [ ]:
!g++ -o cpu cpu.cpp

In [ ]:
%%writefile time.sh
echo "Calculating execution time:"
echo "--------------------------------------"
start=$(date +%s%3N); ./cpu 0 100 100000000000; end=$(date +%s%3N); echo "Temps d'éxecution: $((end-start)) milliseconds"
echo "--------------------------------------"

Overwriting time.sh


In [ ]:
!bash time.sh

Calculating execution time:
--------------------------------------
The integral of the polynomial in the range [0, 100] is: 5000
Temps d'éxecution: 5043 milliseconds
--------------------------------------


# CUDA Implementation

In [ ]:
%%writefile gpu.cu
#include <iostream>
#include <cstdlib>

#define TAILLE_BLOC 1024

__device__ double polynome(double x) {
    return x;
}

__global__ void calculerIntegral(double a, double b, int num_intervals, double *resultat) {
    // Calcule la largeur de chaque sous-intervalle
    double h = (b - a) / num_intervals;
    // Initialise la somme partielle
    double somme = 0.5 * (polynome(a) + polynome(b));

    // Calcule l'index global du thread
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int pas = blockDim.x * gridDim.x;

    // Initialise la somme partielle du bloc
    double somme_partielle = 0.0;
    for (int i = idx; i < num_intervals; i += pas) {
        double x = a + (i + 0.5) * h;
        somme_partielle += polynome(x);
    }

    // Réduction dans le bloc
    __shared__ double memoire_partagee[TAILLE_BLOC];
    memoire_partagee[threadIdx.x] = somme_partielle;
    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            memoire_partagee[threadIdx.x] += memoire_partagee[threadIdx.x + stride];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        // Échelle pour l'opération atomique atomicCAS
        unsigned long long int* adresse_comme_i = reinterpret_cast<unsigned long long int*>(resultat);
        unsigned long long int ancienne_valeur = *adresse_comme_i;
        unsigned long long int nouvelle_valeur;
        do {
            ancienne_valeur = *adresse_comme_i;
            nouvelle_valeur = __double_as_longlong(__longlong_as_double(ancienne_valeur) + memoire_partagee[0] * h);
        } while (atomicCAS(adresse_comme_i, ancienne_valeur, nouvelle_valeur) != ancienne_valeur);
    }
}

int main(int argc, char* argv[]) {
    if (argc < 4) {
        std::cerr << "Usage: " << argv[0] << " <a> <b> <num_intervalles>" << std::endl;
        return 1;
    }

    double a = std::atof(argv[1]);
    double b = std::atof(argv[2]);
    int num_intervalles = std::atoi(argv[3]);
    double integral = 0.0;

    double *d_resultat;
    cudaMalloc(&d_resultat, sizeof(double));
    cudaMemcpy(d_resultat, &integral, sizeof(double), cudaMemcpyHostToDevice);

    int numThreads = TAILLE_BLOC;
    int numBlocs = (num_intervalles + numThreads - 1) / numThreads;
    calculerIntegral<<<numBlocs, numThreads>>>(a, b, num_intervalles, d_resultat);

    cudaError_t cuda_error = cudaGetLastError();
    if (cuda_error != cudaSuccess) {
        std::cerr << "Erreur de lancement du noyau CUDA: " << cudaGetErrorString(cuda_error) << std::endl;
        cudaFree(d_resultat);
        return 1;
    }

    cudaMemcpy(&integral, d_resultat, sizeof(double), cudaMemcpyDeviceToHost);
    cudaFree(d_resultat);

    std::cout << "L'intégrale du polynôme dans l'intervalle [" << a << ", " << b << "] est: " << integral << std::endl;

    return 0;
}

Overwriting gpu.cu


In [ ]:
!nvcc -o gpu gpu.cu

In [ ]:
%%writefile time.sh
echo "Calculating execution time:"
echo "--------------------------------------"
start=$(date +%s%3N); ./gpu 0 100 100000000000; end=$(date +%s%3N); echo "Temps d'éxecution: $((end-start)) milliseconds"
echo "--------------------------------------"

Overwriting time.sh


In [ ]:
!bash time.sh

Calculating execution time:
--------------------------------------
L'intégrale du polynôme dans l'intervalle [0, 100] est: 5000
Temps d'éxecution: 654 milliseconds
--------------------------------------
